<a href="https://colab.research.google.com/github/vedantdave77/project.Orca/blob/master/Project/project-Dog_Breed_CNN_Classification/CNN_Classifier(PyTorch).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CNN From Scratch 

---

### Embeding Google Drive for Data Access



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [69]:
# set parameters which I usually used during project 
norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
img_short_side_resize = 256
img_input_size = 224
shuffle = True
num_workers = 16
batch_size = 64

## Data : [Access/ Transform/ Load]

Reference: 
1. [Data Loader](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

2. [Customize Data Set Operation](https://pytorch.org/docs/stable/torchvision/datasets.html)

3. [Data Transform](https://pytorch.org/docs/stable/torchvision/transforms.html?highlight=transform)


In [70]:
# import required libraries 
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from PIL import Image
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True                                          # cutting image (specially, short of edge)

import torch
import torchvision.transforms as transforms
from torchvision import datasets 

# define data_transformation and batch_size 
transform_train = transforms.Compose([
                                      transforms.Resize(img_short_side_resize),
                                      transforms.ColorJitter(brightness = 0.2, contrast = 0.2, saturation =0.2, hue = 0.1),
                                      transforms.RandomHorizontalFlip(),
                                      transforms.RandomResizedCrop(img_input_size, scale=(0.08,1), ratio = (1,1)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean = norm_mean, std = norm_std)
                                       ])

transform_test = transforms.Compose([
                                     transforms.Resize(img_input_size),
                                     transforms.FiveCrop(img_input_size),
                                     transforms.Lambda(lambda crops: torch.stack([
                                                                                  transforms.Compose([
                                                                                                      transforms.ToTensor(),
                                                                                                      transforms.Normalize(mean = norm_mean, std = norm_std)])(crop) for crop in crops
                                                                                  ]))

                                    ])

# load data (define datasets)
train_data = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/train/",transform_train)
valid_data = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/valid/",transform_test)
test_data  = datasets.ImageFolder("/content/drive/My Drive/Data /dogImages/test/",transform_test)

# separate imput and labels(classes)
data = {"train" : train_data, "valid" : valid_data, "test" : test_data}
n_classes = len(train_data.classes)

# create loaders (train, valid, test)
train_loader = torch.utils.data.DataLoader(data["train"], batch_size = batch_size, num_workers = num_workers, shuffle = shuffle, pin_memory = True)
valid_loader = torch.utils.data.DataLoader(data["valid"], batch_size = int(np.floor(batch_size/5)), num_workers=0, shuffle = shuffle, pin_memory = True) 
test_loader = torch.utils.data.DataLoader(data["test"], batch_size = int(np.floor(batch_size/5)), num_workers=0, shuffle = shuffle, pin_memory = True)

# loader dictionary
loaders_dict = {"train" : train_loader, "valid" : valid_loader, "test" : test_loader}

## Model Architecture :
### Create CNN Classifier 


In [71]:
import torch.nn as nn
import torch.nn.functional as F

# generate the CNN Architecture from Scratch
class CNN(nn.Module):
    def __init__(self,n_classes,layer1_depth = 32):
        super(CNN, self).__init__()
        
        # define layer wise depth
        layer2_depth = layer1_depth *2                                         # 32 --> 64
        layer3_depth = layer2_depth *2                                         # 64 --> 128

        # define Max-pooling layer
        self.pool = nn.MaxPool2d(2,2)

        # convolution set 1 
        self.conv1_1 = nn.Conv2d(3, layer1_depth, 3, stride=1, padding=1)
        self.conv1_2 = nn.Conv2d(layer1_depth, layer1_depth, 3, stride=1, padding=1)
        self.conv1_3 = nn.Conv2d(layer1_depth, layer1_depth, 3, stride=1, padding=1)
        self.bn1_1 = nn.BatchNorm2d(layer1_depth)
        self.bn1_2 = nn.BatchNorm2d(layer1_depth)
        self.bn1_3 = nn.BatchNorm2d(layer1_depth)

        # convolution set 2 
        self.conv2_1 = nn.Conv2d(layer1_depth, layer2_depth, 3, stride=1, padding=1)
        self.conv2_2 = nn.Conv2d(layer2_depth, layer2_depth, 3, stride=1, padding=1)
        self.conv2_3 = nn.Conv2d(layer2_depth, layer2_depth, 3, stride=1, padding=1)
        self.bn2_1 = nn.BatchNorm2d(layer2_depth)
        self.bn2_2 = nn.BatchNorm2d(layer2_depth)
        self.bn2_3 = nn.BatchNorm2d(layer2_depth)

        # convolution set 3 
        self.conv3_1 = nn.Conv2d(layer2_depth, layer3_depth, 3, stride=1, padding=1)
        self.conv3_2 = nn.Conv2d(layer3_depth, layer3_depth, 3, stride=1, padding=1)
        self.conv3_3 = nn.Conv2d(layer3_depth, layer3_depth, 3, stride=1, padding=1)
        self.bn3_1 = nn.BatchNorm2d(layer3_depth)
        self.bn3_2 = nn.BatchNorm2d(layer3_depth)
        self.bn3_3 = nn.BatchNorm2d(layer3_depth)

        # output 
        self.output = nn.Linear(layer3_depth,n_classes)                         # 128 ---> 133

        # Initialize weight
        nn.init.kaiming_normal_(self.conv1_1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv1_2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv1_3.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2_1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2_2.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv2_3.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv3_1.weight, nonlinearity='relu')
        nn.init.kaiming_normal_(self.conv3_2.weight, nonlinearity='relu') 
        nn.init.kaiming_normal_(self.conv3_3.weight, nonlinearity='relu')       




    def forward(self,x):
        # Conv Flow 1
        x = F.relu(self.bn1_1(self.conv1_1(x)))
        x = F.relu(self.bn1_2(self.conv1_2(x)))
        x = F.relu(self.bn1_3(self.conv1_3(x)))
        x = self.pool(x)

        # Conv Flow 2
        x = F.relu(self.bn2_1(self.conv2_1(x)))
        x = F.relu(self.bn2_2(self.conv2_2(x)))
        x = F.relu(self.bn2_3(self.conv2_3(x)))
        x = self.pool(x)

        # Conv Flow 3
        x = F.relu(self.bn3_1(self.conv3_1(x)))
        x = F.relu(self.bn3_2(self.conv3_2(x)))
        x = F.relu(self.bn3_3(self.conv3_3(x)))
        x = self.pool(x)

        # fuse the dimension (height=2, width =3)
        x = x.view(x.size(0),x.size(1),-1)
        x = x.max(2)[0]

        # output 
        x = self.output(x)
        
        return x

In [72]:
Conv_model = CNN(n_classes)

use_cuda = torch.cuda.is_available()
if not use_cuda:
    print('CUDA is not available.  Training on CPU ...')
    device = "cpu"
else:
    print('CUDA is available!  Training on GPU ...')
    device = torch.device("cuda:0")
    print("Using",torch.cuda.get_device_name(device))
    

if use_cuda:
    Conv_model.cuda()           

CUDA is available!  Training on GPU ...
Using Tesla K80


In [73]:
# Now for the learning rate. Explanation below. 
learning_rates = 5e-4 * np.logspace(0,1.5,9)                                    # 9 values between log( 0 to 1.5 ) * 5^-4
learning_rate = learning_rates[2]

## Specify loss function and optimizer 



In [74]:
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau 

# loss function 
criterion = nn.CrossEntropyLoss()

# optimizer 
optimizer = optim.Adam(Conv_model.parameters(),learning_rate)

# learning_rate shedular 
scheduler = ReduceLROnPlateau(optimizer,'min',verbose=True)


## Train and Validate Model

In [75]:
import time 
def train_epoch(model,train_loader,optimizer,criterion,device):
    train_loss = 0.0
    model.train()                                                               # define training job
    for batch_idx, (data,target) in enumerate(train_loader):                                                                     
        data, target = data.to(device),target.to(device)                        # move to CUDA
        optimizer.zero_grad()                                                   # reset gradient 
        output = model(data)                                                    # run model and get output 
        loss = criterion(output, target)                                        # calculate loss
        train_loss += loss.item() * data.size(0)                                # calculate gradients
        loss.backward()                                                         # update each layer para value
        optimizer.step()
    train_loss = train_loss/len(train_loader.dataset)
    return model, train_loss

In [76]:
def valid_epoch(model,valid_loader,criterion,device,fivecrop): 
    valid_loss = 0.0
    model.eval()
    with torch.no_grad():
        for data, target in valid_loader:
            data, target = data.to(device), target.to(device)                   # tranfer to CUDA Tesla P 100
            if fivecrop == "mean":
                bs, ncrops, c, h, w = data.size()                               # dimenstion due to 5 crop method              
                output = model(data.view(-1,c,h,w))
                output = output.view(bs, ncrops, -1).mean(1)
            elif fivecrop == "max":
                bs, ncrops, c,h,w = data.size()
                output = model(data.view(-1,c,h,w))
                output = output.view(bs, ncrops, -1).max(1)[0] 
            else:
                output = model(data)
            
            loss = criterion(output, target)                                    # update losses
            valid_loss += loss.item() * data.size(0)
    valid_loss = valid_loss/len(valid_loader.dataset)
    return valid_loss    

In [61]:
def train(n_epochs, loaders_dict, model,optimizer, criterion, device, path_model, fivecrop = None, lr_scheduler = None):
    valid_loss_min = np.Inf
    train_loss = []
    valid_loss = []
    # time everything 
    time_start = time.time()
    for epoch in range(1, n_epochs+1): 
        time_start_epoch = time.time()
        
        model, train_loss_epoch = train_epoch(model,loaders_dict["train"],optimizer,criterion,device)
        train_loss.append(train_loss_epoch)

        # validate this epoch 
        valid_loss_epoch = valid_epoch(model,loaders_dict["valid"],criterion, device, fivecrop)

        # call learning rate scheduler 
        if lr_scheduler is not None:
            lr_scheduler.step(valid_loss_epoch)
        valid_loss.append(valid_loss_epoch)
        
        # save model for new lowest validation loss 
        if valid_loss_epoch <= valid_loss_min:
            torch.save(model.state_dict(),path_model)
            valid_loss_min = valid_loss_epoch

        print("Epoch {} done in {:.2f} seconds. \t | Training Loss: {:.3f} \t | Validation Loss: {:.3f}".format(
            epoch, time.time() - time_start_epoch, train_loss_epoch, valid_loss_epoch))
    print(f"{n_epochs} epochs ready in {(time.time() - time_start):.3f} seconds. Minimum validation loss: {valid_loss_min:.3f}")
    model.load_state_dict(torch.load(path_model))
    return model
        
# give real training action 
Conv_model_1 = train(100, loaders_dict, Conv_model, optimizer, criterion, device, 'Conv_model.pt', fivecrop = "mean", lr_scheduler = scheduler)

Epoch 1 done in 206.86 seconds. 	 | Training Loss: 4.577 	 | Validation Loss: 4.606
Epoch 2 done in 202.94 seconds. 	 | Training Loss: 4.499 	 | Validation Loss: 4.576
Epoch 3 done in 207.09 seconds. 	 | Training Loss: 4.454 	 | Validation Loss: 4.425
Epoch 4 done in 200.14 seconds. 	 | Training Loss: 4.393 	 | Validation Loss: 4.345
Epoch 5 done in 208.64 seconds. 	 | Training Loss: 4.350 	 | Validation Loss: 4.314
Epoch 6 done in 205.22 seconds. 	 | Training Loss: 4.290 	 | Validation Loss: 4.342
Epoch 7 done in 203.23 seconds. 	 | Training Loss: 4.228 	 | Validation Loss: 4.176
Epoch 8 done in 199.71 seconds. 	 | Training Loss: 4.167 	 | Validation Loss: 4.248
Epoch 9 done in 203.20 seconds. 	 | Training Loss: 4.106 	 | Validation Loss: 4.048
Epoch 10 done in 206.10 seconds. 	 | Training Loss: 4.034 	 | Validation Loss: 4.178
Epoch 11 done in 200.50 seconds. 	 | Training Loss: 3.955 	 | Validation Loss: 3.974
Epoch 12 done in 204.41 seconds. 	 | Training Loss: 3.859 	 | Validation L

In [65]:
# load best validation accuracy model
Conv_model_1.load_state_dict(torch.load("Conv_model.pt"))

<All keys matched successfully>

In [78]:
def test(loaders, model, criterion, device):
    # monitor test loss and accuracy
    test_loss = 0.
    correct = 0.
    total = 0.
    model.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(loaders['test']):
            # move to GPU
            data, target = data.to(device), target.to(device)
            bs, ncrops, c, h, w = data.size()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(data.view(-1, c, h, w)) # fuse batch size and ncrops
            output = output.view(bs, ncrops, -1).mean(1)        
            # calculate the loss
            loss = criterion(output, target)
            # update average test loss 
            test_loss = test_loss + ((1 / (batch_idx + 1)) * (loss.data - test_loss))
            # convert output probabilities to predicted class
            pred = output.data.max(1, keepdim=True)[1]
            # compare predictions to true label
            correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
            total += data.size(0)            
    print('Test Loss: {:.6f}\n'.format(test_loss))
    print('\nTest Accuracy: %2d%% (%2d/%2d)' % (
        100. * correct / total, correct, total))

In [79]:
test(loaders_dict, Conv_model, criterion, device)

Test Loss: 5.919703


Test Accuracy:  0% ( 5/836)


In [80]:
pip freeze > requirements.txt

In [63]:
# keep learning, Enjoy Empowering